In [14]:
import chromadb
import pandas as pd
import os

# 1. Configuration
db_path = os.path.abspath(r"D:\pY\InsuranceRAG\local_db")
print(f"📂 Looking for database at: {db_path}")

client = chromadb.PersistentClient(path=db_path)
collections = client.list_collections()
collection_names = [c.name for c in collections]
print(f"📚 Found {len(collections)} collections: {collection_names}")

# 2. Function to inspect a specific collection
def inspect_collection_dynamic(name):
    if name not in collection_names:
        print(f"❌ Collection '{name}' does not exist.")
        return None
    
    collection = client.get_collection(name=name)
    results = collection.get(include=["documents", "metadatas"])
    
    if not results['ids']:
        print(f"⚠️ Collection '{name}' exists but is empty.")
        return None
        
    records = []
    for i in range(len(results['ids'])):
        # 1. Start with the core fields (ID and Document Preview)
        entry = {
            "ID": results['ids'][i],
            "Content Preview": results['documents'][i][:100].replace('\n', ' ') + "..."
        }
        
        # 2. Dynamically add all metadata fields
        meta = results['metadatas'][i] if results['metadatas'] else {}
        for key, value in meta.items():
            # If the value is a file path, just show the filename for neatness
            if key == "source" and isinstance(value, str):
                entry["Source File"] = os.path.basename(value)
            else:
                entry[key] = value
                
        records.append(entry)
    
    # 3. Create DataFrame (Pandas handles the alignment of different keys automatically)
    df = pd.DataFrame(records)
    
    # Optional: Move 'ID' and 'Content Preview' to the end for better readability
    cols = [c for c in df.columns if c not in ["ID", "Content Preview"]] + ["ID", "Content Preview"]
    return df[cols]

# 3. Fetch and Display Both Collections
print("\n--- 📜 MASTER POLICIES ---")
df_policies = inspect_collection_dynamic("policy_master_collection")
if df_policies is not None:
    display(df_policies.sort_values(by="Source File"))

print("\n--- 📑 CLIENT CLAIMS ---")
df_claims = inspect_collection_dynamic("claims_collection")
if df_claims is not None:
    display(df_claims)

print("\n--- 📑 EVALUATION AUDIT LOG ---")
df_audit = inspect_collection_dynamic("evaluation_audit_log")
if df_audit is not None:
    display(df_audit)


# 4. Summary Table
if df_policies is not None or df_claims is not None:
    print("\n📊 Database Summary:")
    for c in collections:
        print(f" - {c.name}: {c.count()} chunks")

📂 Looking for database at: D:\pY\InsuranceRAG\local_db
📚 Found 3 collections: ['claims_collection', 'evaluation_audit_log', 'policy_master_collection']

--- 📜 MASTER POLICIES ---


,creationdate,creationDate,moddate,producer,source_type,submission_date,Source File,title,keywords,page,...,subject,total_pages,client_id,format,author,file_path,creator,chunk_method,ID,Content Preview
0,2024-02-29T16:03:12+04:00,D:20240229160312+04'00',2024-02-29T16:03:13+04:00,Adobe PDF Library 17.0,Policy,N/A,Motor Perfect T&C_UAE_EN.pdf.ingesting,,,0,...,,23,Company,PDF 1.7,,D:\pY\InsuranceRAG\storage\policies\Motor\2026...,Adobe InDesign 19.2 (Windows),semantic,1f00355a-594c-43e2-8e5f-6c06f7c815f5,1 CAR INSURANCE Policy handbook Motor Perfect...
33,2024-02-29T16:03:12+04:00,D:20240229160312+04'00',2024-02-29T16:03:13+04:00,Adobe PDF Library 17.0,Policy,N/A,Motor Perfect T&C_UAE_EN.pdf.ingesting,,,12,...,,23,Company,PDF 1.7,,D:\pY\InsuranceRAG\storage\policies\Motor\2026...,Adobe InDesign 19.2 (Windows),semantic,ba0555e7-bb37-483b-ae21-ca20474aa074,c. As to the damages to items and properties...
34,2024-02-29T16:03:12+04:00,D:20240229160312+04'00',2024-02-29T16:03:13+04:00,Adobe PDF Library 17.0,Policy,N/A,Motor Perfect T&C_UAE_EN.pdf.ingesting,,,13,...,,23,Company,PDF 1.7,,D:\pY\InsuranceRAG\storage\policies\Motor\2026...,Adobe InDesign 19.2 (Windows),semantic,0510561f-5bb8-42c1-934d-986b511e1d00,Policy Handbook Motor Perfect 26 27 obligatio...
35,2024-02-29T16:03:12+04:00,D:20240229160312+04'00',2024-02-29T16:03:13+04:00,Adobe PDF Library 17.0,Policy,N/A,Motor Perfect T&C_UAE_EN.pdf.ingesting,,,13,...,,23,Company,PDF 1.7,,D:\pY\InsuranceRAG\storage\policies\Motor\2026...,Adobe InDesign 19.2 (Windows),semantic,6e9f826d-91af-4a2a-9b9b-72e48e0e4fae,"2. The accidents that have occurred, caused, ..."
36,2024-02-29T16:03:12+04:00,D:20240229160312+04'00',2024-02-29T16:03:13+04:00,Adobe PDF Library 17.0,Policy,N/A,Motor Perfect T&C_UAE_EN.pdf.ingesting,,,13,...,,23,Company,PDF 1.7,,D:\pY\InsuranceRAG\storage\policies\Motor\2026...,Adobe InDesign 19.2 (Windows),semantic,0f161c0f-d967-4f08-9025-4a26a0c918e5,If the Motor Vehicle is used outside the road...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,2024-02-29T16:03:12+04:00,D:20240229160312+04'00',2024-02-29T16:03:13+04:00,Adobe PDF Library 17.0,Policy,N/A,Motor Perfect T&C_UAE_EN.pdf.ingesting,,,10,...,,23,Company,PDF 1.7,,D:\pY\InsuranceRAG\storage\policies\Motor\2026...,Adobe InDesign 19.2 (Windows),semantic,a4a5ad3d-dcf2-4355-b9fd-0ad3ab1f9960,a. With respect to a fleet insurance policy ...
27,2024-02-29T16:03:12+04:00,D:20240229160312+04'00',2024-02-29T16:03:13+04:00,Adobe PDF Library 17.0,Policy,N/A,Motor Perfect T&C_UAE_EN.pdf.ingesting,,,10,...,,23,Company,PDF 1.7,,D:\pY\InsuranceRAG\storage\policies\Motor\2026...,Adobe InDesign 19.2 (Windows),semantic,b876d717-9857-4615-b7b4-bf721f57eecb,"On the occurrence of an accident, the Company..."
28,2024-02-29T16:03:12+04:00,D:20240229160312+04'00',2024-02-29T16:03:13+04:00,Adobe PDF Library 17.0,Policy,N/A,Motor Perfect T&C_UAE_EN.pdf.ingesting,,,11,...,,23,Company,PDF 1.7,,D:\pY\InsuranceRAG\storage\policies\Motor\2026...,Adobe InDesign 19.2 (Windows),semantic,fd63e1df-1520-4642-a87a-fdb1988651c9,Policy Handbook Motor Perfect 22 23 road-wort...
15,2024-02-29T16:03:12+04:00,D:20240229160312+04'00',2024-02-29T16:03:13+04:00,Adobe PDF Library 17.0,Policy,N/A,Motor Perfect T&C_UAE_EN.pdf.ingesting,,,6,...,,23,Company,PDF 1.7,,D:\pY\InsuranceRAG\storage\policies\Motor\2026...,Adobe InDesign 19.2 (Windows),semantic,b73e519c-d65f-4264-bd0e-c9e3f303aa5d,Policy Handbook Motor Perfect 12 13 of the me...



--- 📑 CLIENT CLAIMS ---
⚠️ Collection 'claims_collection' exists but is empty.

--- 📑 EVALUATION AUDIT LOG ---


,client_id,submission_date,instance_id,start_time,status,risk_score,completion_time,ID,Content Preview
0,125ds,25-Jan-2026,664aa0de-5229-40c4-b1a6-f719dd6543bb,NaN,NaN,NaN,NaN,cf97d820-14ff-489c-930c-1b3b1596341a,"Verdict: Based on the provided information, I'..."
1,12345,20-Jan-2026,1924fd68-909e-4507-91c4-d6667c7b0252,2026-02-10T17:57:13.075851,In_Progress,NaN,NaN,dedf0b5e-6b16-4d8d-9d48-ee39df4ab858,Investigation started for client 12345....
2,12345,20-Jan-2026,1924fd68-909e-4507-91c4-d6667c7b0252,NaN,Completed,0.0,2026-02-10T17:59:14.179507,2b0c5fd0-98ab-4341-8897-970e0a3709a7,Verdict: **APPROVED** The final verdict is AP...
3,98634,20-Jan-2026,3371202c-dd02-475e-8ec1-494bc7818f35,2026-02-10T18:05:36.506066,In_Progress,NaN,NaN,faed0fc2-16de-453c-b355-0110b581f705,Investigation started for client 98634....
4,98634,20-Jan-2026,3371202c-dd02-475e-8ec1-494bc7818f35,NaN,Completed,0.0,2026-02-10T18:07:42.782694,aaf5b51f-451b-4a63-8f60-fb544807b01a,Verdict: Based on the provided context and req...



📊 Database Summary:
 - claims_collection: 0 chunks
 - evaluation_audit_log: 5 chunks
 - policy_master_collection: 63 chunks
